In [2]:
import pandas as pd
import numpy as np

In [3]:
ground_truth = pd.read_csv("dataset/GPP/final_results.csv", index_col=False)
columns = ground_truth.columns
columns = ["Player"]+columns.to_list()[1:]
ground_truth.columns = columns
labels = ["Achiever","Killer","Socializer","Explorer","Casual","Hardcore"]
print(ground_truth.columns)
columns = ["graph"] + [f"{x}_pred" for x in labels] + [f"{x}_true" for x in labels]
model_results = pd.read_csv("tsp_result.csv", index_col=False)
model_results.columns = columns
print(model_results)

Index(['Player', 'Total de Partidas', 'Bartle Classification',
       'Gamer Dedication', 'Bicalho's Classification', 'Partida_1',
       'Partida_2', 'Partida_3', 'Partida_4', 'Partida_5', 'Partida_6',
       'Partida_7', 'Partida_8', 'Partida_9', 'Partida_10'],
      dtype='object')
                                            graph  Achiever_pred  Killer_pred  \
0    info_10.4.2021;20.2_60722e6a3dfd1c6bdca7dfd3            0.0          1.0   
1   info_10.5.2021;18.31_6099a625fe3e0d4c21c84668            0.0          1.0   
2   info_10.5.2021;14.17_60996a9c5be66eccea739f98            0.0          1.0   
3   info_21.2.2021;18.25_6032cfc70d82a8c24f640a79            0.0          1.0   
4   info_10.5.2021;15.49_609980374560af5f47dac71e            1.0          1.0   
..                                            ...            ...          ...   
93  info_24.4.2021;18.48_60849234d3a29ae0504e66f5            1.0          1.0   
94    info_9.5.2021;0.52_60975c790297290d85f1be91            1.0  

In [14]:
player2graphs = {}
for _, row in ground_truth.iterrows():
    player = row["Player"]
    if player not in player2graphs:
        player2graphs[player] = []
    for i in range(1,11):
        graph = row[f"Partida_{i}"]
        if graph is not np.nan:
            player2graphs[player].append(graph)
print(player2graphs)

{'VICTORARRUDAPEREIRA@GMAIL.COM': ['info_10.5.2021;12.21_60994f68575f2291b32b51d9.xml', 'info_10.5.2021;12.20_60994f3107c9aa5ed0194616.xml', 'info_10.5.2021;12.18_60994eb607c9aa5ed0194615.xml', 'info_10.5.2021;12.21_60994f83575f2291b32b51da.xml'], 'MARCELLOBNF@ALUNO.PUC-RIO.BR': ['info_24.4.2021;18.48_6084921ad3a29ae0504e66f4.xml', 'info_24.4.2021;18.47_60849208d3a29ae0504e66f3.xml', 'info_24.4.2021;18.48_60849234d3a29ae0504e66f5.xml'], 'ARTUR.ASSUNCAOPINHEIRO@GMAIL.COM': ['info_12.5.2021;22.30_609c81503d1684c72dbe3808.xml', 'info_12.5.2021;22.33_609c82011e0e91fb4e9de967.xml', 'info_12.5.2021;22.32_609c81961e0e91fb4e9de966.xml'], 'PABLOMMUNIZ@GMAIL.COM': ['info_10.5.2021;23.23_6099ea8642934df9a92936c9.xml', 'info_10.5.2021;23.16_6099e90471373b5a5f65f8cf.xml', 'info_10.5.2021;23.19_6099e9b042934df9a92936c7.xml', 'info_10.5.2021;23.15_6099e8d171373b5a5f65f8ce.xml', 'info_10.5.2021;23.18_6099e97742934df9a92936c6.xml', 'info_10.5.2021;23.17_6099e93e42934df9a92936c5.xml', 'info_10.5.2021;23

In [15]:
from collections import Counter
def get_repeated_mode(lst):
    if (len(lst)) == 0:
        return ""
    data = Counter(lst)
    most_common = data.most_common()
    result = [most_common[0]]
    del most_common[0]
    i = 0
    while len(most_common) > 0 and i < len(most_common) and result[-1][1] == most_common[i][1]:
        result.append(most_common[i])
        i = i+1
    string_result = [x[0] for x in result]
    if len(string_result) > 1:
        return max(string_result, key=len)
    else:
        return string_result[0]
print(get_repeated_mode(["1","1","2","2","2","3","1","4","4"]))

player2class = {}
f = lambda lst: max(set(lst), key=lst.count)
for player, graphs in player2graphs.items():
    if player not in player2class:
        player2class[player] = {
            "Bartle": [],
            "Dedication": []
        }
    for graph in graphs:
        result_row = model_results[model_results["graph"] == graph[:-4]]
        if result_row.empty:
            continue
        bartle_preds = [label for label in labels[:4] if result_row[f"{label}_pred"].iloc[0] > 0]
        dedica_preds = [label for label in labels[4:] if result_row[f"{label}_pred"].iloc[0] > 0]
        player2class[player]["Bartle"].append("/".join(bartle_preds))
        player2class[player]["Dedication"].append("/".join(dedica_preds))
    player2class[player]["Bartle"] = get_repeated_mode(player2class[player]["Bartle"])
    player2class[player]["Dedication"] = get_repeated_mode(player2class[player]["Dedication"])
print(player2class)

1
{'VICTORARRUDAPEREIRA@GMAIL.COM': {'Bartle': 'Achiever/Killer', 'Dedication': 'Casual/Hardcore'}, 'MARCELLOBNF@ALUNO.PUC-RIO.BR': {'Bartle': 'Killer', 'Dedication': 'Hardcore'}, 'ARTUR.ASSUNCAOPINHEIRO@GMAIL.COM': {'Bartle': 'Killer', 'Dedication': 'Hardcore'}, 'PABLOMMUNIZ@GMAIL.COM': {'Bartle': 'Killer', 'Dedication': 'Hardcore'}, 'VICTORFACO@GMAIL.COM': {'Bartle': 'Achiever/Killer', 'Dedication': 'Hardcore'}, 'AMANDACNOLIVEIRA@GMAIL.COM': {'Bartle': 'Achiever/Killer', 'Dedication': 'Hardcore'}, 'KILLERASUS@GMAIL.COM': {'Bartle': '', 'Dedication': ''}, 'STEFANYCAMACHO1007@GMAIL.COM': {'Bartle': 'Killer', 'Dedication': 'Hardcore'}, 'DAMAZIO1214@GMAIL.COM': {'Bartle': 'Killer', 'Dedication': 'Hardcore'}, 'MCMELO@GMX.COM': {'Bartle': 'Achiever/Killer', 'Dedication': 'Hardcore'}, 'RAQUELAZEVEDOOLIVEIRASILVA@GMAIL.COM': {'Bartle': 'Killer', 'Dedication': 'Hardcore'}, 'GUSTAVOABSAMPAIO@GMAIL.COM': {'Bartle': '', 'Dedication': ''}, 'EDUPONTESDESIGN@GMAIL.COM': {'Bartle': 'Achiever/Killer'

In [16]:
model_pred_df = pd.DataFrame.from_dict(player2class, orient='index').reset_index().rename({
    "index":"Player"}, axis=1)
model_pred_df = model_pred_df[model_pred_df['Bartle'] != '']
model_pred_df.rename(columns={
    "Bartle" : "PING Bartle",
    "Dedication" : "PING Dedication"
}, inplace=True)
print(model_pred_df)

                                  Player      PING Bartle  PING Dedication
0          VICTORARRUDAPEREIRA@GMAIL.COM  Achiever/Killer  Casual/Hardcore
1           MARCELLOBNF@ALUNO.PUC-RIO.BR           Killer         Hardcore
2       ARTUR.ASSUNCAOPINHEIRO@GMAIL.COM           Killer         Hardcore
3                  PABLOMMUNIZ@GMAIL.COM           Killer         Hardcore
4                   VICTORFACO@GMAIL.COM  Achiever/Killer         Hardcore
5             AMANDACNOLIVEIRA@GMAIL.COM  Achiever/Killer         Hardcore
7           STEFANYCAMACHO1007@GMAIL.COM           Killer         Hardcore
8                  DAMAZIO1214@GMAIL.COM           Killer         Hardcore
9                         MCMELO@GMX.COM  Achiever/Killer         Hardcore
10  RAQUELAZEVEDOOLIVEIRASILVA@GMAIL.COM           Killer         Hardcore
12             EDUPONTESDESIGN@GMAIL.COM  Achiever/Killer  Casual/Hardcore
13              JOAOPEDRO.JPBF@GMAIL.COM  Achiever/Killer         Hardcore
14               ENZOANTO

In [17]:
ground_truth_redux = ground_truth.iloc[:,:5]
merged_df = ground_truth_redux.merge(model_pred_df, on="Player")
merged_df = merged_df[merged_df["Bicalho's Classification"]!="Not available"]
merged_df["Bicalho Bartle"] = merged_df["Bicalho\'s Classification"].apply(lambda x: x.split(" ")[1])
merged_df["Bicalho Dedication"] = merged_df["Bicalho\'s Classification"].apply(lambda x: x.split(" ")[0])
merged_df.drop(columns={
    "Total de Partidas",
    "Bicalho\'s Classification"
}, inplace=True)
print(merged_df)

                                  Player Bartle Classification  \
0          VICTORARRUDAPEREIRA@GMAIL.COM                Killer   
2       ARTUR.ASSUNCAOPINHEIRO@GMAIL.COM                Killer   
3                  PABLOMMUNIZ@GMAIL.COM       Achiever/Killer   
4                   VICTORFACO@GMAIL.COM                Killer   
7                  DAMAZIO1214@GMAIL.COM       Achiever/Killer   
8                         MCMELO@GMX.COM                Killer   
9   RAQUELAZEVEDOOLIVEIRASILVA@GMAIL.COM       Achiever/Killer   
10             EDUPONTESDESIGN@GMAIL.COM              Achiever   
11              JOAOPEDRO.JPBF@GMAIL.COM       Achiever/Killer   
12               ENZOANTONELLO@GMAIL.COM                Killer   
13              ENRICO.V.GNANI@GMAIL.COM                Killer   
20           GABRIELLUIZ.NEVES@GMAIL.COM                Killer   
21             KARLAHCAMACHO@HOTMAIL.COM                Killer   
22                    JOAO@ESCARLATE.COM                Killer   
23     LUI

In [20]:
from sklearn.preprocessing import MultiLabelBinarizer
bartle_mlb = MultiLabelBinarizer()
bartle_mlb.fit([{"Achiever","Killer"}])
dedica_mlb = MultiLabelBinarizer()
dedica_mlb.fit([{"Hardcore", "Casual"}])
bartle_gt_encoded = bartle_mlb.transform([x.split("/") for x in list(merged_df["Bartle Classification"])])
dedica_gt_encoded = dedica_mlb.transform([x.split("/") for x in list(merged_df["Gamer Dedication"])])
bartle_pg_encoded = bartle_mlb.transform([x.split("/") for x in list(merged_df["PING Bartle"])])
dedica_pg_encoded = dedica_mlb.transform([x.split("/") for x in list(merged_df["PING Dedication"])])
bartle_bc_encoded = bartle_mlb.transform([x.split("/") for x in list(merged_df["Bicalho Bartle"])])
dedica_bc_encoded = dedica_mlb.transform([x.split("/") for x in list(merged_df["Bicalho Dedication"])])
print(dedica_gt_encoded, dedica_pg_encoded)

[[0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]] [[1 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 1]
 [0 1]
 [0 1]
 [0 1]
 [1 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]


In [19]:
from sklearn.metrics import classification_report

print("Bartle Classification - Bicalho's Classification")
print(classification_report(bartle_gt_encoded, bartle_bc_encoded, target_names=bartle_mlb.classes_))
print("Bartle Classification - PINGUMIL Classification")
print(classification_report(bartle_gt_encoded, bartle_pg_encoded, target_names=bartle_mlb.classes_))
print("Gamer Dedication - Bicalho's Classification")
print(classification_report(dedica_gt_encoded, dedica_bc_encoded, target_names=dedica_mlb.classes_))
print("Gamer Dedication - PINGUMIL Classification")
print(classification_report(dedica_gt_encoded, dedica_pg_encoded, target_names=dedica_mlb.classes_))

Bartle Classification - Bicalho's Classification
              precision    recall  f1-score   support

    Achiever       0.00      0.00      0.00         8
      Killer       0.81      0.87      0.84        15

   micro avg       0.72      0.57      0.63        23
   macro avg       0.41      0.43      0.42        23
weighted avg       0.53      0.57      0.55        23
 samples avg       0.72      0.58      0.63        23

Bartle Classification - PINGUMIL Classification
              precision    recall  f1-score   support

    Achiever       0.20      0.25      0.22         8
      Killer       0.83      1.00      0.91        15

   micro avg       0.61      0.74      0.67        23
   macro avg       0.52      0.62      0.57        23
weighted avg       0.61      0.74      0.67        23
 samples avg       0.64      0.78      0.65        23

Gamer Dedication - Bicalho's Classification
              precision    recall  f1-score   support

      Casual       0.25      0.25      0.2

In [23]:
bartle_model_results = pd.read_csv("semisup_tsp_results_bartle.csv", index_col=False)
bartle_model_results.columns = columns[:5]+columns[7:11]
#print(bartle_model_results)
dedica_model_results = pd.read_csv("semisup_tsp_results_dedication.csv", index_col=False)
dedica_model_results.columns = columns[:1]+columns[5:7]+columns[11:13]
#print(dedica_model_results)
model_results = bartle_model_results.merge(dedica_model_results, on="graph")
model_results = model_results[columns]
print(model_results)

                                            graph  Achiever_pred  Killer_pred  \
0    info_10.5.2021;12.0_60994a8b575f2291b32b51d6            1.0          1.0   
1    info_10.4.2021;20.2_60722e6a3dfd1c6bdca7dfd3            1.0          1.0   
2   info_10.5.2021;18.31_6099a625fe3e0d4c21c84668            1.0          1.0   
3   info_10.5.2021;14.17_60996a9c5be66eccea739f98            1.0          1.0   
4   info_21.2.2021;18.25_6032cfc70d82a8c24f640a79            1.0          1.0   
..                                            ...            ...          ...   
94  info_24.4.2021;18.48_60849234d3a29ae0504e66f5            1.0          1.0   
95    info_9.5.2021;0.52_60975c790297290d85f1be91            1.0          1.0   
96  info_11.5.2021;16.12_609ad72af91ed9390fdbf5db            1.0          1.0   
97   info_24.4.2021;20.8_6084a5022ddb62b60b9161bf            1.0          1.0   
98   info_10.5.2021;12.6_60994c0207c9aa5ed0194612            1.0          1.0   

    Socializer_pred  Explor